In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


name_schema = StructType(fields= [
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)])

drivers_schema = StructType(fields= [
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("name", name_schema, True),
    StructField("dob", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)])
drivers_df = spark.read \
    .schema(drivers_schema) \
    .json("abfss://bronze@mjuanworkshopetl.dfs.core.windows.net/drivers.json")

drivers_df.display()

In [0]:
drivers_df.printSchema()

In [0]:
from pyspark.sql.functions import col, lit, concat, current_timestamp

drivers_renamed_df =  drivers_df.withColumnRenamed("driverId", "driver_id") \
                                .withColumnRenamed("driverRef", "driver_ref") \
                                .withColumn("ingestion_date", current_timestamp()) \
        .withColumn('name', concat(col('name.forename'), lit(" "), col('name.surname'))) \
        .drop(col('url'))

drivers_renamed_df.display()

In [0]:
drivers_renamed_df.write.mode('overwrite').option("overwriteSchema", "true").saveAsTable('formula1_project.silver.drivers_transformed')